In [6]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd


In [35]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'tiny-imagenet-5_2'

train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train'])
val_data = datasets.ImageFolder(os.path.join(data_dir, 'val'), data_transforms['val'])

# Load the validation labels.
val_annotations = pd.read_csv(os.path.join(data_dir, 'val', 'val_annotations.txt'), sep='\t', names=['name', 'label', '1', '2', '3', '4'])

# Assign labels to validation data.
label_dict = dict()
for i, img in enumerate(val_data.imgs):
    imgname = os.path.basename(img[0])
    label = val_annotations[val_annotations['name'] == imgname]['label'].tolist()[0]
    label_int = label_dict.setdefault(label, len(label_dict))
    val_data.imgs[i] = (img[0], label_int)

image_datasets = {'train':train_data, 'val':val_data}    

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=5,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

use_gpu = torch.cuda.is_available()

In [39]:
for data in dataloaders['val']:
    inputs, labels = data
    print(labels)


 3
 2
 2
 0
 1
[torch.LongTensor of size 5]


 1
 0
 0
 2
 3
[torch.LongTensor of size 5]


 2
 0
 3
 0
 4
[torch.LongTensor of size 5]


 3
 0
 3
 3
 2
[torch.LongTensor of size 5]


 3
 1
 3
 2
 4
[torch.LongTensor of size 5]


 0
 2
 1
 3
 2
[torch.LongTensor of size 5]


 3
 0
 4
 3
 1
[torch.LongTensor of size 5]


 0
 4
 1
 1
 4
[torch.LongTensor of size 5]


 1
 2
 1
 2
 0
[torch.LongTensor of size 5]


 1
 3
 3
 1
 4
[torch.LongTensor of size 5]


 1
 0
 0
 0
 1
[torch.LongTensor of size 5]


 1
 0
 3
 4
 2
[torch.LongTensor of size 5]


 4
 1
 2
 4
 4
[torch.LongTensor of size 5]


 3
 4
 4
 2
 2
[torch.LongTensor of size 5]


 0
 0
 4
 3
 4
[torch.LongTensor of size 5]


 3
 1
 3
 4
 0
[torch.LongTensor of size 5]


 1
 2
 2
 2
 1
[torch.LongTensor of size 5]


 1
 1
 3
 1
 3
[torch.LongTensor of size 5]


 0
 0
 1
 4
 3
[torch.LongTensor of size 5]


 3
 3
 0
 1
 4
[torch.LongTensor of size 5]


 1
 3
 0
 2
 1
[torch.LongTensor of size 5]


 4
 4
 0
 3
 1
[torch.LongTensor 

In [8]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=50):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0] * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [13]:
# 1. using a pre-trained CNN as a fixed feature extractor
model_conv = torchvision.models.resnet34(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 5)

if use_gpu:
    model_conv = model_conv.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opoosed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=50)

Epoch 0/49
----------


Process Process-19:
Process Process-20:
Process Process-17:
Process Process-18:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/mlx/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/mlx/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/mlx/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/mlx/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/mlx/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/mlx/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/mlx/anaconda3/lib/python3.6/multiprocessing/process.p

KeyboardInterrupt: 

In [16]:
model_ft = models.resnet34(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 5)

if use_gpu:
    model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Train the model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /Users/mlx/.torch/models/resnet34-333f7ec4.pth
100.0%


In [35]:
torch.save(model_ft.state_dict(),'/Users/mlx/Documents/SpringSemester/DataScience/assignment3/model/model_ft')